In [ ]:
def variable_summaries(var, namescope_this):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization).
       Tensor can be the activity of hidden layer.Monitor the saturation.
    """

    with tf.name_scope(namescope_this):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)

def create_placeholders(n_H0, n_W0, n_C0, n_y, namescope_this):
    """
    Creates the placeholders for the tensorflow session.
    
    Arguments:
    n_H0 -- scalar, height of an input image
    n_W0 -- scalar, width of an input image
    n_C0 -- scalar, number of channels of the input
    n_y -- scalar, number of classes
        
    Returns:
    X -- placeholder for the data input, of shape [None, n_H0, n_W0, n_C0] and dtype "float"
    Y -- placeholder for the input labels, of shape [None, n_y] and dtype "float"
    """

    ### START CODE HERE ### (≈2 lines)
    X = tf.placeholder(tf.float32, shape = (None, n_H0, n_W0, n_C0))
    Y = tf.placeholder(tf.float32, shape = (None, n_y))
    ### END CODE HERE ###
    
    return X, Y


    
    return parameters
def conv_layer(X, f, strides, layer_name, act=tf.nn.relu, use_cudnn_on_gpu=False):
    """
    Reusable code for making a simple neural net layer.
    It does a matrix multiply, bias add, and then uses relu to nonlinearize.
    It also sets up name scoping so that the resultant graph is easy to read,
    and adds a number of summary ops.

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window [20, 8, 1, 64]
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    Y -- output of the identity block, tensor of shape (n_H, n_W, n_C)

    """
    
    with tf.name_scope(layer_name):
        W = tf.get_variable('W', f, initializer= tf.contrib.layers.xavier_initializer())
        B = tf.get_variable('bias', f[3], initializer= tf.zeros(f[3]))
        Z = tf.nn.conv2d(X, W, strides, 'VALID', use_cudnn_on_gpu = use_cudnn_on_gpu, name = 'Z') + B
        A = act(Z, name = 'Activation')
        
        variable_summaries(W, 'W')
        variable_summaries(B, 'B')
        variable_summaries(Z, 'WX+B')
        variable_summaries(A, 'activation')
        
    return A
    
def create_model_conv2_with_name_scope(fingerprint_input, model_settings, is_training):
    parameters['W2'] = [10, 4, 64, 64]
    parameters['F_strides2'] = [1,1,1,1]
    parameters['M2'] = [1, 2, 2, 1]
    parameters['M_strides2'] = [1, 2, 2, 1]

    parameters['W3'] = [5, 2, 64, 128]
    parameters['F_strides3'] = [1,1,1,1]
    parameters['M3'] = [1, 2, 2, 1]
    parameters['M_strides3'] = [1, 2, 2, 1]

    parameters['W3'] = [5, 2, 128, 128]
    parameters['F_strides3'] = [1,1,1,1]
    parameters['M3'] = [1, 2, 2, 1]
    parameters['M_strides3'] = [1, 2, 2, 1]

    parameters['W4'] = [5, 2, 128, 128]
    parameters['F_strides4'] = [1,1,1,1]
    
    
    
    if is_training:
        dropout_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    parameters = model_settings['parameter']
    # reshape input
    input_frequency_size = model_settings['dct_coefficient_count']
    input_time_size = model_settings['spectrogram_length']
    fingerprint_4d = tf.reshape(fingerprint_input,
                              [-1, input_time_size, input_frequency_size, 1])
    
    ## first layer.
    layer_name = 'layer_1'
    hidden1 = conv_layer(fingerprint_4d, parameters['W1'], parameters['F_strides1'], layer_1_name, act=tf.nn.relu, use_cudnn_on_gpu=False)
    with tf.name_scope(layer_name):
        if is_training:
            hidden1_dropout = tf.nn.dropout(hidden1, dropout_prob)
        else:
            hidden1_dropout = hidden1
    
        maxpool1= tf.nn.max_pool(hidden1_dropout,  ksize = parameters['M1'], strides = parameters['M_strides1'], padding = 'SAME')

    
    # second layer
    layer_name = 'layer_2'
    hidden2 = conv_layer(maxpool1, parameters['W2'], parameters['F_strides2'], layer_name, act=tf.nn.relu, use_cudnn_on_gpu=False)
    with tf.name_scope(layer_name):
        if is_training:
            hidden2_dropout = tf.nn.dropout(hidden2, dropout_prob)
        else:
            hidden2_dropout = hidden2
    
        maxpool2= tf.nn.max_pool(hidden2_dropout,  ksize = parameters['M2'], strides = parameters['M_strides2'], padding = 'SAME')

   ## third layer.
    layer_name = 'layer_3'
    hidden3 = conv_layer(maxpool2, parameters['W3'], parameters['F_strides3'], layer_name, act=tf.nn.relu, use_cudnn_on_gpu=False)
    with tf.name_scope(layer_name):
        if is_training:
            hidden3_dropout = tf.nn.dropout(hidden3, dropout_prob)
        else:
            hidden3_dropout = hidden3
        
        maxpool3= tf.nn.max_pool(hidden3_dropout,  ksize = parameters['M3'], strides = parameters['M_strides3'], padding = 'SAME')
      
    layer_name = 'layer_4'
    hidden4 = conv_layer(hidden3_dropout, parameters['W4'], parameters['F_strides4'], layer_name, act=tf.nn.relu, use_cudnn_on_gpu=False)
    with tf.name_scope(layer_name):
        if is_training:
            hidden4_dropout = tf.nn.dropout(hidden4, dropout_prob)
        else:
            hidden4_dropout = hidden4
    
    ## hidden4 is used for fully connected layer.
    layer_name = 'fully_connected'
    with tf.name_scope( layer_name)
        label_count = model_settings['label_count']
        last_layer = tf.contrib.layers.flatten(hidden4_dropout)
        final_fc = tf.contrib.layers.fully_connected(last_layer, label_count, activation_fn=None)

    if is_training:
        return final_fc, dropout_prob
    else:
        return final_fc